In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np
import os
import cv2

import sys
sys.path.append('..')

from shared.data import KITTIData, draw_matches, VisualOdometry

%matplotlib widget

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
# https://github.com/matplotlib/ipympl

In [ ]:
DATASET_DIR = os.path.join('../', 'data/KITTI/dataset')
dataset = KITTIData(DATASET_DIR)

In [ ]:
l_img, r_img = dataset.get_color_images(20)
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=[15,9])
ax1.imshow(l_img)
ax2.imshow(r_img)

# ORB features 

In [ ]:
idx = 25
c_img, _ = dataset.get_color_images(idx)
n_img, _ = dataset.get_color_images(idx+1)

orb = cv2.ORB_create(
    nfeatures=500,
    scaleFactor=1.2,
    nlevels=8,
    edgeThreshold=31,
    firstLevel=0,
    WTA_K=2,
    scoreType=cv2.ORB_HARRIS_SCORE,
    patchSize=31,
    fastThreshold=10
)
c_kp, c_des = orb.detectAndCompute(c_img, None)
n_kp, n_des = orb.detectAndCompute(n_img, None)

c_img_canvas = c_img.copy()
n_img_canvas = n_img.copy()
cv2.drawKeypoints(c_img_canvas, c_kp, c_img_canvas, color=(0,255,0), flags=0)
cv2.drawKeypoints(n_img_canvas, n_kp, n_img_canvas, color=(0,255,0), flags=0)

fig, (ax1, ax2) = plt.subplots(2, 1, figsize=[15,9])
ax1.imshow(c_img_canvas)
ax2.imshow(n_img_canvas)

# Harris corners

In [ ]:
feature_params = dict(maxCorners=150,
                      qualityLevel=0.3,
                      minDistance=7,
                      blockSize=7)

# Parameters for lucas kanade optical flow
lk_params = dict(winSize=(15, 15),
                 maxLevel=2,
                 criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

c_img_gray = cv2.cvtColor(c_img, cv2.COLOR_RGB2GRAY)
n_img_gray = cv2.cvtColor(n_img, cv2.COLOR_RGB2GRAY)

c_feat_corners = cv2.goodFeaturesToTrack(c_img_gray, mask=None, **feature_params)
n_feat_corners, st, err = cv2.calcOpticalFlowPyrLK(c_img_gray, n_img_gray, c_feat_corners, None, **lk_params)

c_feat_corners = c_feat_corners[st==1]
n_feat_corners = n_feat_corners[st==1]

print(c_feat_corners.shape)

c_img_canvas = c_img.copy()
n_img_canvas = n_img.copy()

radius = 5
color = (20, 255, 20)

for p in c_feat_corners:
    cv2.circle(c_img_canvas, tuple(p), radius, color, 2)

for p in n_feat_corners:
    cv2.circle(n_img_canvas, tuple(p), radius, color, 2)

fig, (ax1, ax2) = plt.subplots(2, 1, figsize=[15,9])
ax1.imshow(c_img_canvas)
ax2.imshow(n_img_canvas)

# FAST features

In [ ]:
c_img_gray = cv2.cvtColor(c_img, cv2.COLOR_RGB2GRAY)
n_img_gray = cv2.cvtColor(n_img, cv2.COLOR_RGB2GRAY)

lk_params = dict(winSize=(15, 15),
                 maxLevel=2,
                 criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

fast = cv2.FastFeatureDetector_create(
    threshold=20,
    nonmaxSuppression=True,
    type=cv2.FAST_FEATURE_DETECTOR_TYPE_9_16
)
c_feats = fast.detect(c_img_gray, None)
c_feats = [[list(cf.pt)] for cf in c_feats]
c_feats = np.array(c_feats, dtype=np.float32)
n_feats, st, err = cv2.calcOpticalFlowPyrLK(c_img_gray, n_img_gray, c_feats, None, **lk_params)

c_feats = c_feats[st==1]
n_feats = n_feats[st==1]

print(c_feats.shape, n_feats.shape)

c_img_canvas = c_img.copy()
n_img_canvas = n_img.copy()

radius = 5
color = (20, 20, 255)

for p in c_feats:
    cv2.circle(c_img_canvas, tuple(p), radius, color, 2)

for p in n_feats:
    cv2.circle(n_img_canvas, tuple(p), radius, color, 2)

fig, (ax1, ax2) = plt.subplots(2, 1, figsize=[15,9])
ax1.imshow(c_img_canvas)
ax2.imshow(n_img_canvas)

# Matching

In [ ]:
bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
matches = bf.match(c_des, n_des)
matches = sorted(matches, key = lambda x:x.distance)

# Filter with distance larger than 35
matches = [m for m in matches if m.distance < 35]

# img_matched = cv2.drawMatches(c_img, c_kp, n_img, n_kp, matches[:10], None, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
img_matched = draw_matches(c_img, c_kp, n_img, n_kp, matches)

plt.figure(figsize=(15,9))
plt.imshow(img_matched)